In [ ]:
import os
import numpy as np
import pandas as pd
os.chdir('idle')
from importlib import reload
import shuhua
import soyeon
import yuqi
import minnie
import miyeon
import soojin

In [ ]:
#If you need the list of ALL TICKS, here you go:

a = pd.read_excel('../docs/bigset.xlsx',index_col='SYM')
ticks = list(a.index)

In [ ]:
# reload(yuqi)
# reload(minnie)
# reload(soojin)

#Making data sets for a ML Party

TICK = 'MSFT'

RATIOS = minnie.macro_scrape(TICK,'financial-ratios')

income = minnie.macro_scrape(TICK,'income-statement')
shares = income.loc['Shares Outstanding']*1000000
cash = minnie.macro_scrape(TICK,'cash-flow-statement')


ratios_calc = yuqi.calc_ratios(TICK,RATIOS,5,5)
prices = minnie.macro_prices(TICK,shares)

YRRAY = soojin.make_y(prices,income,cash,RATIOS)

YRRAY[:5]